In [1]:
from typing import Dict, TypedDict, Optional
from langgraph.graph import StateGraph, END
import os
import sys

# Add the parent directory of 'utilities' to the Python path
sys.path.append(r'C:\Users\Ai Sukmoren\Desktop\chatbot')
from utilities.llms import llm

In [16]:
class GraphState(TypedDict):
    question: Optional[str] = None
    classification: Optional[str] = None
    response: Optional[str] = None

workflow = StateGraph(GraphState)


def classify(question):
    return llm.invoke(f"classify intent of given input as greeting or not_greeting. Output just the class. Input:{question}").content

def classify_input_node(state):
    question = state.get('question', '').strip()
    classification = classify(question)
    return {"classification": classification}

def handle_greeting_node(state):
    return {"response": "Hello! How can I help you today?"}

def handle_llm(state):
    question = state.get('question', '').strip()
    return {"response": llm.invoke(question).content}

def bye(state):
    current_response = state.get('response')
    return {"response": current_response + ". The graph has finished"}

workflow.add_node("classify_input", classify_input_node)
workflow.add_node("handle_greeting", handle_greeting_node)
workflow.add_node("handle_llm", handle_llm)
workflow.add_node("bye", bye)


In [17]:
workflow.set_entry_point("classify_input")
workflow.add_edge('handle_greeting', END)
workflow.add_edge("handle_llm", "bye")
workflow.add_edge('bye', END)

In [18]:
def decide_next_node(state):
    return "handle_greeting" if state.get('classification') == "greeting" else "handle_llm"

workflow.add_conditional_edges(
    "classify_input",
    decide_next_node,
    {
        "handle_greeting": "handle_greeting",
        "handle_llm": "handle_llm"
    }
)

In [19]:
app = workflow.compile()
app.invoke({'question': 'what is 2 + 2'})

{'question': 'what is 2 + 2',
 'classification': 'not_greeting',
 'response': '2 + 2 equals 4.. The graph has finished'}

In [ ]:
app.get_graph().print_ascii()


           +-----------+               
           | __start__ |               
           +-----------+               
                  *                    
                  *                    
                  *                    
        +----------------+             
        | classify_input |             
        +----------------+             
            ..        ..               
          ..            ..             
         .                ..           
+------------+              .          
| handle_llm |              .          
+------------+              .          
       *                    .          
       *                    .          
       *                    .          
    +-----+       +-----------------+  
    | bye |       | handle_greeting |  
    +-----+*      +-----------------+  
            **        **               
              **    **                 
                *  *                   
            +---------+                
